In [ ]:
import json
import numpy as np
import os
import pandas as pd
from ensemble_boxes import *

#csv file -> ensemble -> csv file
def ensemble(csv_data, save_dir, weights=[2, 1, 1], iou_thr=0.2, skip_box_thr=0.0001, sigma=0.1):

    bbox = []
    score = []
    label = []

    save_dir = os.path.dirname(save_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for data in csv_data:
        model_bbox = []
        model_score = []
        model_label = []
        for predict_per_image in data['PredictionString']:
            if(predict_per_image == ''):
                model_bbox.append([])
                model_score.append([])
                model_label.append([])
                continue
            a = predict_per_image.split()
            predict_num = len(a)
            predict_per_image = np.array([a[i:i+6]for i in range(0, predict_num,6)])
            
            bbox_img = predict_per_image[:,2:6].astype(float) / 1024.0
            score_img = predict_per_image[:,1].astype(float)
            label_img = predict_per_image[:,0].astype(int)
            
            model_bbox.append(bbox_img.tolist())
            model_score.append(score_img.tolist())
            model_label.append(label_img.tolist())

            # print(np.array(model_bbox).shape)
            # print(np.array(model_score).shape)
        bbox.append(model_bbox)
        score.append(model_score)
        label.append(model_label)


    submission_num = len(csv_data)
    img_num = len(label[0])
    #(3,4871,100,4) -> (4871,3,100,4) 

    prediction_strings = []
    file_names = []

    for i in range(img_num):

        boxes_list = []
        scores_list = []
        labels_list = []

        for j in range(submission_num):
            boxes_list.append(bbox[j][i])
            scores_list.append(score[j][i])
            labels_list.append(label[j][i])
        
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes = boxes * 1024
        labels = labels.astype(int)

        prediction_string = ''
        for k in range(len(scores)):
            prediction_string += str(labels[k]) + ' ' + str(scores[k]) + ' ' + str(boxes[k][0]) + ' ' + str(boxes[k][1]) + ' ' + str(
                    boxes[k][2]) + ' ' + str(boxes[k][3]) + ' '
        prediction_strings.append(prediction_string)
        image_id = str(i).zfill(4)
        file_names.append('test/' + image_id + '.jpg')

    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv(os.path.join(save_dir, f'submission_ensemble.csv'), index=None)

In [ ]:
#csv 파일 경로 설정
csv_path1 = '../baseline/mmdetection_boostcamp/work_dirs/deformable_detr_trash/submission_latest.csv'
csv_path2 = '../baseline/mmdetection_boostcamp/work_dirs/detr_r50_8x2_150e_trash/submission_epoch_80.csv'
csv_path3 = '../baseline/Swin-Transformer-Object-Detection/work_dirs/Swin_trash/submission_best_bbox_mAP.csv'

csv_data1 = pd.read_csv(csv_path1,keep_default_na=False)
csv_data2 = pd.read_csv(csv_path2,keep_default_na=False)
csv_data3 = pd.read_csv(csv_path3,keep_default_na=False)

csv_data = [csv_data1, csv_data2,csv_data3]

save_dir = './'

weights = [1, 1, 1]

iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1